# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [59]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [60]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [61]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.Conv = torch.nn.Sequential(torch.nn.Conv2d(1,28,kernel_size=3,padding=1,stride=1),torch.nn.Conv2d(28,64,kernel_size=3,padding=1,stride=1),torch.nn.MaxPool2d(kernel_size=2, stride=2),torch.nn.Conv2d(64,64,kernel_size=3,padding=1,stride=1),torch.nn.Conv2d(64,64,kernel_size=3,padding=1,stride=1),torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.Dense = torch.nn.Linear(7*7*64, 10)   
    def forward(self, input):
        x = self.Conv(input)
        x = x.view(-1, 7*7*64)
        x = self.Dense(x)
        return x
model = SimpleNet()
model = model.cuda()
print(model)

# TODO:define loss function and optimiter
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


SimpleNet(
  (Conv): Sequential(
    (0): Conv2d(1, 28, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(28, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (Dense): Linear(in_features=3136, out_features=10, bias=True)
)


Next, we can start to train and evaluate!

In [62]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    epoch_acc = 0.0

    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        x_train = images
        y_train = labels
        x_train,y_train = Variable(x_train.cuda()),Variable(y_train.cuda())
        y_pred = model(x_train)
        _,pred = torch.max(y_pred, 1)

        optimizer.zero_grad()
        loss = criterion(y_pred, y_train)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += torch.sum(pred == y_train.data)

    epoch_loss = epoch_loss*64/len(train_dataset)
    epoch_acc = epoch_acc.double()/len(train_dataset)
    print("Epoch{}: Loss is:{:.4f},Acc is:{:4f}".format(epoch, epoch_loss, epoch_acc))

        
# evaluate
# TODO:calculate the accuracy using traning and testing dataset

test_acc = 0.0
for images, labels in tqdm(test_loader):
    x_test = images
    y_test = labels
    
    x_test,y_test = x_test.cuda(),y_test.cuda()
    y = model(x_test)
    _,y = torch.max(y,1)
    test_acc += torch.sum(y == y_test.data)

test_acc = test_acc.double()/len(test_dataset)
print("Test_acc is:{:4f}".format(test_acc))



100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:11<00:00, 40.84it/s]


Epoch0: Loss is:0.0841,Acc is:0.946067


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:11<00:00, 41.71it/s]


Epoch1: Loss is:0.0310,Acc is:0.979233


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:11<00:00, 41.00it/s]


...

Epoch8: Loss is:0.0131,Acc is:0.989950


100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:11<00:00, 39.57it/s]


Epoch9: Loss is:0.0118,Acc is:0.990417


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 64.79it/s]


Test_acc is:0.979300


#### Q5:
Please print the training and testing accuracy.